In [1]:
from dataLoadFunction import process_engagement_data
from database_utils import create_table_if_not_exists
from database_utils import insert_data
import os
import psycopg2
import datetime
import pandas as pd

In [5]:
file = "./inputData/previousData/20240517 Engagement List.xlsx"

In [6]:
df_raw = pd.read_excel(file)

df_raw.columns = df_raw.columns.str.replace(' ', '_')
print(df_raw.columns)

Index(['Engagement_ID', 'Engagement', 'Creation_Date', 'Release_Date',
       'Plan_Start_Date', 'Plan_End_Date', 'Last_Status_Change_Date',
       'Last_Invoice_Issued_Date', 'Last_Time_Charged_Date',
       'Last_Expenses_Charged_Date',
       ...
       'AR_Aging_31_60_days_Lead', 'AR_Aging_61_90_days_Lead',
       'AR_Aging_91_120_days_Lead', 'AR_Aging_121_150_days_Lead',
       'AR_Aging_151_179_days_Lead', 'AR_Aging_180_365_days_Lead',
       'AR_Aging_Above_365_days_Lead', 'Revenue_Days', 'Billed_Revenue_Days',
       'Unbilled_Revenue_Days'],
      dtype='object', length=312)


In [2]:
df_processed = process_engagement_data(
    "file"
)

INFO:dataLoadFunction:File Path: ./inputData/PreviousWeeksEngagementLists/20240517 Engagement List.xlsx
INFO:dataLoadFunction:Data loaded with shape (rows and columns): (1079, 312)
INFO:dataLoadFunction:Data loading time: 1.20 seconds
INFO:dataLoadFunction:Data reduced to key columns only: (1079, 14)
INFO:dataLoadFunction:Column reduction time: 0.00 seconds
INFO:dataLoadFunction:Data filtered by EP service line and released eng. codes only. Filtered data shape: (1079, 14)
INFO:dataLoadFunction:Added calculated columns, new data shape: (1079, 17) and time taken: 0.01 seconds


In [3]:
df_processed.head()



,engagement_id,creation_date,release_date,last_time_charged_date,last_expenses_charged_date,last_active_etc-p_date,engagement,client,engagement_partner,engagement_partner_gui,engagement_manager,engagement_manager_gui,engagement_partner_service_line,engagement_status,last_etc_date,report_date,etc_age
0,E-66318700,2021-11-01 00:00:00,2021-11-01 00:00:00,2021-11-19 00:00:00,None,2022-08-31 00:00:00,Project Longbow - Pensions,Cinven Capital Management (VII) General Partne...,"Mignault, Matthew",5044822,"Khan, Haroon",2335483,Consulting,Released,2022-08-31,2024-05-17,625
1,E-67675286,2023-09-04 00:00:00,2024-02-27 00:00:00,2024-05-17 00:00:00,None,None,24 Mar USS Audit ITRA,Universities Superannuation Scheme,"Sloey, Louise",5040753,"Aftab, Rao Junaid",65003487,Consulting,Released,2024-02-27,2024-05-17,80
2,E-65556437,2020-02-20 00:00:00,2020-02-20 00:00:00,2023-11-24 00:00:00,2024-05-10 00:00:00,None,ECL+ investment code,Ernst & Young LLP,"Cade, Martin",2032814,"Cade, Martin",2032814,Consulting,Released,2020-02-20,2024-05-17,1548
3,E-68241170,2024-04-25 00:00:00,2024-04-25 00:00:00,None,None,None,MSFT NFUM Employee Listening,Microsoft Corporation,"Savage, Katherine",2068498,"Pugh, Stephanie",2154674,Consulting,Released,2024-04-25,2024-05-17,22
4,E-67373210,2023-04-13 00:00:00,2023-04-28 00:00:00,2023-07-10 00:00:00,2023-08-02 00:00:00,2024-04-11 00:00:00,Sanlam HR Tech and Operations Review,Sanlam Ltd,"Chouffot, James",2133997,"Sukhathankar, Sampada",3396528,Consulting,Released,2024-04-11,2024-05-17,36


In [10]:
connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT"),
        )

table_name = "engagement_data"

# time = current system time
now = datetime.datetime.now()  






In [11]:
create_table_if_not_exists(connection, table_name)


In [15]:
insert_data(connection, df_processed, table_name, now, upload_user="nick")